## Normal Pipeline
1. Quick Explore
2. Check NULL
## NLP Pipeline steps
1. Stop word removal
2. Casing removal
3. Tokenization
4. Stemming (or Lemmatization)
5. Embedding/Vectorization

In [2]:
#  Get CWD
import os
cwd = os.getcwd()
print(f'CWD: {cwd}')
import sys

hadoop_home_path = f'{cwd}/windows/winutils/hadoop-3.0.0'
os.environ['HADOOP_HOME'] = hadoop_home_path
sys.path.append(f'{hadoop_home_path}/bin')

print('HADOOP_HOME ENV var: ', os.environ['HADOOP_HOME'])

CWD: f:\Projects\sparkimental
HADOOP_HOME ENV var:  f:\Projects\sparkimental/windows/winutils/hadoop-3.0.0


In [3]:
# Imports
import findspark
from pyspark import SparkContext
from pyspark.sql.types import *
from pprint import pprint, pformat

In [4]:
# Find spark to make sure pyspark works (note: make sure only :))
findspark.init()
findspark.find()

'f:\\AppSSD\\WorkTools\\Anaconda\\envs\\sparkimental\\lib\\site-packages\\pyspark'

In [5]:
# Setup spark environment
sc = SparkContext.getOrCreate()
print('Spark version: ', sc.version)
print(f'Hadoop version = {sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}')
print('Spark Config:')
pprint(sc.getConf().getAll())


Spark version:  3.1.2
Hadoop version = 3.2.0
Spark Config:
[('spark.driver.host', 'ProdudePredator.mshome.net'),
 ('spark.app.startTime', '1666862767841'),
 ('spark.app.id', 'local-1666862769943'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '60313'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]


In [6]:
# Extra import (move later)
from pyspark import SparkFiles
from pyspark import SQLContext

In [7]:
# add data file from local system to Spark's RDD
path = cwd.replace('\\', '/').replace('f:/','')
full_path = f'file:///{path}/data/animal-crossing.csv'
sc.addFile(full_path)

In [8]:
# Create sql context
sqlContext = SQLContext(sc)

In [9]:
# read file from hdfs/rdd
file_path = SparkFiles.get('animal-crossing.csv')
print('File path in Spark File system: ', file_path)
df = sqlContext.read.csv(path=file_path, header=True, inferSchema=True)

File path in Spark File system:  C:\Users\USER\AppData\Local\Temp\spark-31a43f65-7128-4b5a-8950-5fe3648df162\userFiles-e4460f48-1d09-4d43-8ce7-f9ea58c4cbbe\animal-crossing.csv


In [10]:
df.printSchema()

root
 |-- grade: integer (nullable = true)
 |-- publication: string (nullable = true)
 |-- text: string (nullable = true)
 |-- date: string (nullable = true)



In [11]:
# Type cast
df = df.withColumn('date', df.date.cast(DateType()))
df.printSchema()

root
 |-- grade: integer (nullable = true)
 |-- publication: string (nullable = true)
 |-- text: string (nullable = true)
 |-- date: date (nullable = true)



In [27]:
# Drop useless column
df = df.select(['date', 'text', 'grade'])
print(f'Data shape: {df.count(), len(df.columns)}')
# Show data samples
df.sample(0.1, seed=0).show(10)

Data shape: (107, 3)


In [19]:
# Basic stats
df.describe().show()

+-------+-----------------+-----------+--------------------+
|summary|            grade|publication|                text|
+-------+-----------------+-----------+--------------------+
|  count|              107|        107|                 107|
|   mean| 90.6355140186916|       null|                null|
| stddev|6.114308185868841|       null|                null|
|    min|               70|   3DJuegos|A beautiful, welc...|
|    max|              100|        XGN|“New Horizons mak...|
+-------+-----------------+-----------+--------------------+



In [18]:
# Nan check
from pyspark.sql.functions import isnan, when, count, col

print("Nan checking")
df.select(
    [
        count(
            when(
                isnan(c)
                | col(c).isNull()
                | (col(c) == "")
                | col(c).contains("None")
                | col(c).contains("Null"),
                c,
            )
        ).alias(c)
        for c in ["grade"]
    ]
).show()

df.select(
    [
        count(
            when(
                col(c).contains("None")
                | col(c).contains("NULL")
                | (col(c) == "")
                | col(c).isNull()
                | isnan(c),
                c,
            )
        ).alias(c)
        for c in ["text"]
    ]
).show()


Nan checking
+-----+-----------+
|grade|publication|
+-----+-----------+
|    0|          0|
+-----+-----------+

+----+
|text|
+----+
|   0|
+----+



# NLP

Refs:
1. https://github.com/SurajMalpani/NLP-using-Spark/blob/master/nlp-using-pyspark-ml.ipynb
2. https://stackoverflow.com/questions/53579444/efficient-text-preprocessing-using-pyspark-clean-tokenize-stopwords-stemming
3. 


```Build steps into pipeline after done```


In [30]:
from pyspark.ml.feature import StopWordsRemover, RegexTokenizer, CountVectorizer
from nltk.stem.snowball import SnowballStemmer

df = df.select('text')

In [35]:
regex_tokenizer = RegexTokenizer(inputCol='text', outputCol='words', toLowercase=True)
raw_words_df = regex_tokenizer.transform(df)
raw_words_df.show(5, truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                                             words|
+--------------------------------------------------+--------------------------------------------------+
|Animal Crossing; New Horizons, much like its pr...|[animal, crossing;, new, horizons,, much, like,...|
|Know that if you’re overwhelmed with the world,...|[know, that, if, you’re, overwhelmed, with, the...|
|With a game this broad and lengthy, there’s mor...|[with, a, game, this, broad, and, lengthy,, the...|
|Animal Crossing: New Horizons is everything I h...|[animal, crossing:, new, horizons, is, everythi...|
|Above all else, Animal Crossing: New Horizons i...|[above, all, else,, animal, crossing:, new, hor...|
+--------------------------------------------------+--------------------------------------------------+
only showing top 5 rows



In [40]:
stop_remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtered_df = stop_remover.transform(raw_words_df)
filtered_df.select(['words', 'filtered']).show(5, truncate=50)
# Note how 'a' is removed

+--------------------------------------------------+--------------------------------------------------+
|                                             words|                                          filtered|
+--------------------------------------------------+--------------------------------------------------+
|[animal, crossing;, new, horizons,, much, like,...|[animal, crossing;, new, horizons,, much, like,...|
|[know, that, if, you’re, overwhelmed, with, the...|[know, you’re, overwhelmed, world,, stuck, insi...|
|[with, a, game, this, broad, and, lengthy,, the...|[game, broad, lengthy,, there’s, discuss, fit, ...|
|[animal, crossing:, new, horizons, is, everythi...|[animal, crossing:, new, horizons, everything, ...|
|[above, all, else,, animal, crossing:, new, hor...|[else,, animal, crossing:, new, horizons, unbea...|
+--------------------------------------------------+--------------------------------------------------+
only showing top 5 rows



In [44]:
from pyspark.sql.functions import udf

stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
stemmed_df = filtered_df.withColumn("stemmed", stemmer_udf("filtered"))
stemmed_df.select('filtered', 'stemmed').show(5, truncate=50)


+--------------------------------------------------+--------------------------------------------------+
|                                          filtered|                                           stemmed|
+--------------------------------------------------+--------------------------------------------------+
|[animal, crossing;, new, horizons,, much, like,...|[anim, crossing;, new, horizons,, much, like, p...|
|[know, you’re, overwhelmed, world,, stuck, insi...|[know, you'r, overwhelm, world,, stuck, inside,...|
|[game, broad, lengthy,, there’s, discuss, fit, ...|[game, broad, lengthy,, there, discuss, fit, on...|
|[animal, crossing:, new, horizons, everything, ...|[anim, crossing:, new, horizon, everyth, hope, ...|
|[else,, animal, crossing:, new, horizons, unbea...|[else,, anim, crossing:, new, horizon, unbeat, ...|
+--------------------------------------------------+--------------------------------------------------+
only showing top 5 rows



In [45]:
count_vectorizer = CountVectorizer(inputCol='stemmed', outputCol='embedded')
cv_model = count_vectorizer.fit(stemmed_df)

embedded_df = cv_model.transform(stemmed_df)
embedded_df.select('stemmed', 'embedded').show(5, truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                           stemmed|                                          embedded|
+--------------------------------------------------+--------------------------------------------------+
|[anim, crossing;, new, horizons,, much, like, p...|(1369,[0,1,9,17,23,28,58,73,137,194,283,330,639...|
|[know, you'r, overwhelm, world,, stuck, inside,...|(1369,[1,16,21,48,67,104,128,135,220,230,312,31...|
|[game, broad, lengthy,, there, discuss, fit, on...|(1369,[3,6,9,14,15,16,21,30,31,34,36,45,60,94,9...|
|[anim, crossing:, new, horizon, everyth, hope, ...|(1369,[0,1,2,4,5,6,29,32,57,64,70,92,95,126,175...|
|[else,, anim, crossing:, new, horizon, unbeat, ...|(1369,[0,1,2,3,4,6,7,14,16,18,31,33,35,44,59,72...|
+--------------------------------------------------+--------------------------------------------------+
only showing top 5 rows



### Build the pipeline

In [46]:
# stages = [
#     regex_tokenizer,
#     stop_remover,
#     stemmer,
#     count_vectorizer
# ]

# from pyspark.ml import Pipeline

# preprocess_pipeline = Pipeline(stages=stages)
# pipeModel = preprocess_pipeline.fit(df)
# training = pipeModel.transform(df)
# training.show(5)

# # TODO: make snowball stemmer into a custom pipe stage for spark

TypeError: Cannot recognize a pipeline stage of type <class 'nltk.stem.snowball.SnowballStemmer'>.